# Lab 3 - Spark MLlib

##### "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E"
-Tom M. Mitchell

#### Machine Learning - the science of getting computers to act without being explicitly programmed

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. It consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering (this example!), dimensionality reduction, as well as lower-level optimization primitives and higher-level pipeline APIs.

It divides into two packages:
- spark.mllib contains the original API built on top of RDDs.
- spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.


Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.

http://spark.apache.org/docs/latest/mllib-guide.html

## Online Purchase Recommendations

Learn how to create a recommendation engine using the Alternating Least Squares algorithm in Spark's machine learning library

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/ALS.png' width="70%" height="70%"></img>

### The data

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.  The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

http://archive.ics.uci.edu/ml/datasets/Online+Retail

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/FullFile.png' width="80%" height="80%"></img>

## Create an RDD from the csv data 

### Download the data

In [1]:
!rm 'OnlineRetail.csv.gz' -f
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz

--2016-05-16 14:25:01--  https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 23.235.44.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|23.235.44.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7483128 (7.1M) [application/octet-stream]
Saving to: 'OnlineRetail.csv.gz'

100%[======================================>] 7,483,128   --.-K/s   in 0.1s    

2016-05-16 14:25:02 (74.0 MB/s) - 'OnlineRetail.csv.gz' saved [7483128/7483128]



### Put the csv into an RDD (at first, each row in the RDD is a string which correlates to a line in the csv)

In [2]:
loadRetailData = sc.textFile("OnlineRetail.csv.gz")
print loadRetailData.take(2)

[u'InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country', u'536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom']


## Prepare and shape the data:  "80% of a Data Scientists  job"

#### Remove the header from the RDD and split the string in each row by comma

In [3]:
header = loadRetailData.first()
loadRetailData = loadRetailData.filter(lambda line: line != header).\
                            map(lambda l: l.split(","))

print loadRetailData.take(2)

[[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom'], [u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']]


##### NOTE:  The original file at UCI's Machine Learning Repository has commas in the product description.  Those have been removed to expediate the lab.
#### Only keep rows that have a purchase quantity of greater than 0, a customerID not equal to 0, and a non blank stock code after removing non-numeric characters.

In [4]:
import re

loadRetailData = loadRetailData.filter(lambda l: int(l[3]) > 0\
                                and len(re.sub("\D", "", l[1])) != 0 \
                                and len(l[6]) != 0)

print loadRetailData.take(2)

[[u'536365', u'85123A', u'WHITE HANGING HEART T-LIGHT HOLDER', u'6', u'12/1/10 8:26', u'2.55', u'17850', u'United Kingdom'], [u'536365', u'71053', u'WHITE METAL LANTERN', u'6', u'12/1/10 8:26', u'3.39', u'17850', u'United Kingdom']]


#### Map each line to a row and create a data frame 

In [5]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

#Convert each line to a Row.
loadRetailData = loadRetailData.map(lambda l: Row(inv=int(l[0]),\
                                    stockCode=int(re.sub("\D", "", l[1])),\
                                    description=l[2],\
                                    quant=int(l[3]),\
                                    invDate=l[4],\
                                    price=float(l[5]),\
                                    custId=int(l[6]),\
                                    country=l[7]))

# Infer the schema, and register the DataFrame as a table.
retailDf = sqlContext.createDataFrame(loadRetailData)
print retailDf.printSchema()

retailDf.registerTempTable("retailPurchases")
print sqlContext.sql("SELECT * FROM retailPurchases limit 2").toPandas()

root
 |-- country: string (nullable = true)
 |-- custId: long (nullable = true)
 |-- description: string (nullable = true)
 |-- inv: long (nullable = true)
 |-- invDate: string (nullable = true)
 |-- price: double (nullable = true)
 |-- quant: long (nullable = true)
 |-- stockCode: long (nullable = true)

None
          country  custId                         description     inv  \
0  United Kingdom   17850  WHITE HANGING HEART T-LIGHT HOLDER  536365   
1  United Kingdom   17850                 WHITE METAL LANTERN  536365   

        invDate  price  quant  stockCode  
0  12/1/10 8:26   2.55      6      85123  
1  12/1/10 8:26   3.39      6      71053  


#### Keep only the data we need (custId, stockCode, and rank)

In [6]:
query = """
SELECT 
    custId, stockCode, 1 as purch
FROM 
    retailPurchases 
group 
    by custId, stockCode"""
retailDf = sqlContext.sql(query)

print retailDf.take(2)

[Row(custId=12838, stockCode=22941, purch=1), Row(custId=17968, stockCode=22731, purch=1)]


### Randomly split the data into a testing set (10% of the data), a cross validation set (10% of the data) a training set (80% of the data)

In [7]:
testDf, cvDf, trainDf = retailDf.randomSplit([.1,.1,.8],1)

print trainDf.take(2)
print cvDf.take(2)
print testDf.take(2)

[Row(custId=17968, stockCode=22731, purch=1), Row(custId=17897, stockCode=84558, purch=1)]
[Row(custId=12838, stockCode=22941, purch=1), Row(custId=13468, stockCode=21231, purch=1)]
[Row(custId=16210, stockCode=20977, purch=1), Row(custId=13090, stockCode=22617, purch=1)]


## Build recommendation models

#### Use training DF to train a model with Alternating Least Squares 
Latent Factors / rank<br>
The number of columns in the user-feature and product-feature matricies)<br>
Iterations / maxIter<br>
The number of factorization runs<br>

In [8]:
from pyspark.ml.recommendation import ALS

als1 = ALS(rank=3, maxIter=15,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model1 = als1.fit(trainDf)

als2 = ALS(rank=15, maxIter=3,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model2 = als2.fit(trainDf)

als3 = ALS(rank=15, maxIter=15,userCol="custId",itemCol="stockCode",ratingCol="purch",implicitPrefs=True)
model3 = als3.fit(trainDf)

print "The models has been trained"

The models has been trained


## Test the models

Use the models to predict what the user will rate a certain item.  The closer our model to 1 that our model rates an item a user has already purchased, the better.

#### Evaluate the model with the cross validation dataframe by using the transform function.

Some of the users or purchases in the cross validation data may not have been in the training data.  Let's remove the ones that are not.

In [9]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import BooleanType
customers = set(trainDf.map(lambda line: line.custId).collect())
stock = set(trainDf.map(lambda line: line.stockCode).collect())

print cvDf.count()
cvDf = cvDf.rdd.filter(lambda line: line.stockCode in stock and\
                                           line.custId in customers).toDF()
print cvDf.count()

25869
25844


In [10]:
predictions1 = model1.transform(cvDf)
predictions2 = model2.transform(cvDf)
predictions3 = model3.transform(cvDf)

print predictions1.take(2)
print predictions2.take(2)
print predictions3.take(2)

[Row(custId=14286, stockCode=20831, purch=1, prediction=0.01889532431960106), Row(custId=13949, stockCode=20831, purch=1, prediction=0.018036123365163803)]
[Row(custId=14286, stockCode=20831, purch=1, prediction=0.04725762456655502), Row(custId=13949, stockCode=20831, purch=1, prediction=0.03975377976894379)]
[Row(custId=14286, stockCode=20831, purch=1, prediction=0.0679328441619873), Row(custId=13949, stockCode=20831, purch=1, prediction=0.06638696044683456)]


#### Calculate and print the Mean Squared Error.   For all ratings, subtract the prediction from the actual purchase (1), square the result, and take the mean of all of the squared differences.

In [11]:
meanSquaredError1 = predictions1.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError2 = predictions2.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError3 = predictions3.map(lambda line: (line.purch - line.prediction)**2).mean()
    
print 'Mean squared error = %.4f for our first model' % meanSquaredError1
print 'Mean squared error = %.4f for our second model' % meanSquaredError2
print 'Mean squared error = %.4f for our second model' % meanSquaredError3

Mean squared error = 0.7388 for our first model
Mean squared error = 0.7003 for our second model
Mean squared error = 0.6691 for our second model


#### Confirm the model by testing it with the test data and the best hyperparameters found during cross validation

In [12]:
filteredTestDf = testDf.rdd.filter(lambda line: line.stockCode in stock and\
                                              line.custId in customers).toDF()
predictions4 = model3.transform(filteredTestDf)
meanSquaredError4 = predictions4.map(lambda line: (line.purch - line.prediction)**2).mean()
    
print 'Mean squared error = %.4f for our best model' % meanSquaredError4

Mean squared error = 0.6677 for our best model


# Implement the model

Use the best model to predict items the user will be interested in.

First, create a dataframe in which each row has the user id and an item id.

In [13]:
from pyspark.sql.functions import lit

stock15544 = set(trainDf.filter(trainDf['custId'] == 15544).map(lambda line: line.stockCode).collect())

userItems = trainDf.select("stockCode").distinct().\
            withColumn('custId', lit(15544)).\
            rdd.filter(lambda line: line.stockCode not in stock15544).toDF()

print userItems.take(5)

[Row(stockCode=21231, custId=15544), Row(stockCode=85231, custId=15544), Row(stockCode=22431, custId=15544), Row(stockCode=23231, custId=15544), Row(stockCode=22631, custId=15544)]


Use 'transform' to rate each item.

In [14]:
userItems = model3.transform(userItems)

print userItems.take(5)

[Row(stockCode=20831, custId=15544, prediction=0.004703751299530268), Row(stockCode=21031, custId=15544, prediction=0.010227371007204056), Row(stockCode=21231, custId=15544, prediction=0.058025069534778595), Row(stockCode=21631, custId=15544, prediction=-0.02571825124323368), Row(stockCode=22031, custId=15544, prediction=0.05723518133163452)]


Print the top 5 recommendations.

In [15]:
print userItems.sort("prediction",ascending=False).take(5)

[Row(stockCode=84997, custId=15544, prediction=0.5976683497428894), Row(stockCode=21213, custId=15544, prediction=0.5788490772247314), Row(stockCode=16161, custId=15544, prediction=0.5184972286224365), Row(stockCode=22138, custId=15544, prediction=0.505846381187439), Row(stockCode=22090, custId=15544, prediction=0.49148523807525635)]


Let's look up this user and the recommended product ID's in the excel file...

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/user.png' width="80%" height="80%"></img>

This user seems to have purchased a lot of childrens gifts and some holiday items.  The recomendation engine we created suggested some items along these lines

##### The ALS algorithm uses some randomness, so the recommendations yours produces may be different than these.

In [17]:
query = """
SELECT 
    distinct description 
FROM 
    retailPurchases 
WHERE 
    stockCode in (84997,21213,16161,22138,22090)
"""
items = sqlContext.sql(query)
print items.toPandas()

                           description
0                   WRAP ENGLISH ROSE 
1    CHILDRENS CUTLERY POLKADOT GREEN 
2              PAPER BUNTING RETROSPOT
3                     WRAP  PINK FLOCK
4                    WRAP BAD HAIR DAY
5    RED 3 PIECE RETROSPOT CUTLERY SET
6    PINK 3 PIECE POLKADOT CUTLERY SET
7          PACK OF 72 SKULL CAKE CASES
8        BAKING SET 9 PIECE RETROSPOT 
9      CHILDRENS CUTLERY POLKADOT PINK
10               WRAP SUKI AND FRIENDS
11  GREEN 3 PIECE POLKADOT CUTLERY SET
12     CHILDRENS CUTLERY POLKADOT BLUE
13   BLUE 3 PIECE POLKADOT CUTLERY SET
14    CHILDRENS CUTLERY RETROSPOT RED 


##### Data Citation
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).